In [2]:
import json
import requests
from furl import furl
from math import ceil
import pandas as pd
import matplotlib
import pyperclip as cp
import pprint
import time
import heapq
import base64
import six
import six.moves.urllib.parse as urllibparse

ALBUM_PLAY_URL = "https://t4ils.dev:4433/api/beta/albumPlayCount"

%matplotlib inline
import matplotlib.pyplot as plt

In [4]:
def get_auth_code(url):
    start = url.index('?')
    end = url.index('&')
    code = url[start + 6:end]
    return code

def create_auth_header(client_id, client_secret):
    auth_header = base64.b64encode(six.text_type(client_id + ':' + client_secret).encode('ascii'))
    return {'Authorization': 'Basic %s' % auth_header.decode('ascii')}

In [5]:
username = 'deeaw2ybwljjvzzv7sftosnqi'
scope = "user-top-read user-read-recently-played user-library-read user-library-modify user-read-playback-state user-read-currently-playing user-modify-playback-state"
client_id = 'e88d92c779fb4f2baf5f69a919db89ec'
client_secret = 'e27b14511a554ea58a2b42d3b6ef5b15'
redirect_uri = 'https://example.com/callback/'
state = 'ilovetocodelol'

auth_code_url = "https://accounts.spotify.com/authorize"
auth_code_dict = {"client_id": client_id, "response_type": "code", "redirect_uri": redirect_uri, "state": state, "scope": scope, "show_dialog": "false"}

auth_code_params = urllibparse.urlencode(auth_code_dict)
                    
get_response = requests.get(auth_code_url, params=auth_code_params)
if get_response.status_code != 200:
    print("Error:", get_response.reason)
    
print(get_response.url)

https://accounts.spotify.com/login?continue=https%3A%2F%2Faccounts.spotify.com%2Fauthorize%3Fclient_id%3De88d92c779fb4f2baf5f69a919db89ec%26response_type%3Dcode%26redirect_uri%3Dhttps%253A%252F%252Fexample.com%252Fcallback%252F%26state%3Dilovetocodelol%26scope%3Duser-top-read%2Buser-read-recently-played%2Buser-library-read%2Buser-library-modify%2Buser-read-playback-state%2Buser-read-currently-playing%2Buser-modify-playback-state%26show_dialog%3Dfalse


In [6]:
token_url = "https://accounts.spotify.com/api/token"
get_response_url = "https://example.com/callback/?code=AQB7OJ-8SbUIVWmewssY6lXxFzHtHkFe1Hwhi3dQ13BB1CBLilcBzgLvHvFB24mK1uhmto193U9fPU_monBcVOf6jxzrum1dLghFJek1lcvEW07VlYlcRZwpgXe-s-qAAHoAyN6Y4NkGdUDKFNma-TfaT6RmysWW71C0MLLiMPyCqmfwgP4b5U0fAks8UJFuN3iiFwM1jP8mlcpYMwKBPj31weBnjx3FKhxCYIRIwXjZ_B__qpwYNlrDrqDTQD-YyTcCUJQ5IN2Jt7qSPMIab3wM461H_2wEdnENB3GILLyACoBWhY5fmStgRT9X45flvsrWbXXzE2GtRHr2O2BIzJqfsm8Ovly7Al9tg6xaa4WM-O3rZ-xAwMI_PIXLqJ2tzot3hQnkngTwgIaAcxzc7vMJ8hvqhmSYPSI&state=ilovetocodelol"
token_data = {"redirect_uri": redirect_uri, "code": get_auth_code(get_response_url), "grant_type": "authorization_code", "scope": scope}
token_headers = create_auth_header(client_id, client_secret)

post_response = requests.post(token_url, data=token_data, headers=token_headers)

if post_response.status_code != 200:
    print("Error:", post_response.reason)

ACCESS_TOKEN = json.loads(post_response.text)["access_token"]
print(ACCESS_TOKEN)

BQAVhqaiXaHAOZ-SjmpqGH2Zl1t-m6SIZnDs8TmdPQo02-zVXCTnUWiyCD-C8zIpmOyWqjjudPojH07NE6eujSLmHPlcD7ZWYcDpQkykjoxAdl8eoQQzsHmstae-3Aw3ImiZZtAJqpxUM-0irI4CMo_Z-FGjmpJRqnyFy_E_RQ6r6vQ8cn0w6vjtfTtx6fukp90OZ5oVpA


In [7]:
url = "https://api.spotify.com/v1/me/tracks"
headers = {'Authorization': "Bearer {}".format(ACCESS_TOKEN)}
tracks = requests.get(url, headers=headers)
parsed = json.loads(tracks.text)
cp.copy(tracks.text)

count_songs = parsed["total"]
print("Number of songs: {}".format(count_songs))

Number of songs: 822


In [8]:
all_songs = []

for i in range(int(ceil(count_songs/50.0))):
    offset = 50*i
    url = "https://api.spotify.com/v1/me/tracks?limit=50&offset={}".format(offset)
    track_headers = {'Authorization': "Bearer {}".format(ACCESS_TOKEN)}
    tracks = requests.get(url, headers=track_headers)
    cp.copy(tracks.text)
    parsed = json.loads(tracks.text)

    all_songs.extend(parsed['items'])
    
try:
    if (len(all_songs) == count_songs):
        print("No responses failed.")
except:
    print("Response from API failed.")
    
print("Number of gathered songs: {}".format(len(all_songs)))

No responses failed.
Number of gathered songs: 822


In [12]:
songs_all = [(all_songs[i]['track']['popularity'], all_songs[i]['track']['name']) for i in range(len(all_songs))]
songs_under_5 = [(all_songs[i]['track']['popularity'], all_songs[i]['track']['name']) for i in range(len(all_songs)) if all_songs[i]['track']['duration_ms'] <= 300000]    
album_ids = {(all_songs[i]['track']['album']['name'], all_songs[i]['track']['album']['id']) 
             for i in range(len(all_songs))}

print("Total songs:", len(songs_all))
print("Total songs under 5 minutes:", len(songs_under_5))
# print(album_ids)

Total songs: 822
Total songs under 5 minutes: 737


In [13]:
largest = heapq.nsmallest(50, songs_under_5)
largest = [(key, value) for value, key in largest]
for song in largest:
    print(song)

('Flyaway', 0)
('High for This', 0)
('Might Not', 0)
('Nonchalant', 0)
('Starboy', 0)
('What You Need', 0)
('10AM/Save The World (feat. Gucci Mane)', 1)
('Lesbian (feat. Gunna & Young Thug)', 1)
('Only 1 (Interlude) (with Travis Scott)', 1)
('Up To Something (feat. Travis Scott & Young Thug)', 1)
('Wake Up', 1)
('No Complaints (feat. Offset & Drake) (Bonus)', 2)
('Only You (feat. WizKid, Offset & J Balvin)', 2)
('Borrowed Love (feat. Swae Lee & WizKid)', 3)
('Final Fantasy', 3)
('No More (feat. Travis Scott, Kodak Black & 21 Savage)', 3)
("Don't Come Out The House (with 21 Savage)", 4)
('Dreamcatcher (feat. Swae Lee & Travis Scott)', 4)
('I Thought About Killing You', 4)
('Is There More', 4)
('No Mistakes', 4)
('Overdue (with Travis Scott)', 4)
('Ratchet Happy Birthday', 4)
('8 Out Of 10', 5)
('Blue Tint', 5)
('Jaded', 5)
('Peak', 5)
('Sandra’s Rose', 5)
('Summer Games', 5)
('Survival', 5)
('Talk Up', 5)
('That’s How You Feel', 5)
("Wouldn't Leave", 5)
('After Dark', 6)
('Can’t Take A 

In [14]:
for album in album_ids:
    print(album)

('ASTROWORLD', '41GuZcammIkupMPKH2OJ6I')
('!', '4YEhTvqsptB5D1n095QL5r')
('Snow White', '6tvVlwo0u8wBCmRvLCJIXQ')
('Suncity', '4UNwL1B7JoymNUiTFJa52B')
('Blank Face LP', '0YbpATCIng8Fz2JrfHmEf7')
('DAMN.', '4eLPsYPBmXABThSJ821sqY')
('Una Mattina', '4aSt14A98gEpCUpKecrl4V')
('Walking On A Dream', '04gYcIojJt78nYnN5oOrKt')
('Black Panther The Album Music From And Inspired By', '3pLdWdkj83EYfDN6H2N8MR')
('Marie Curie', '4m99RhYGgvz4CYOtVtXTaz')
('End Credits', '5bbk7Mo2OiD6q5Bb52DIkC')
('Better', '6Nd4PYvQ7aYid1Gn6sWgeG')
('Views', '0G1ffjfFuTUTVjDrVdLimH')
('i think you think too much of me', '7AVvQhnDEUidKyJsXmQ7ju')
('Ramreaper', '3bz6yWBKeXKvSRpeJ6NyUn')
('Floating', '73UyBWt2xMqRuP8RnbtgAH')
('Future & Juice WRLD Present... WRLD ON DRUGS', '6P9PZjWXoCRF5b66BafPKY')
('Nothing Was The Same (Deluxe)', '2gXTTQ713nCELgPOS0qWyt')
('Trilogy', '5EbpxRwbbpCJUepbqVTZ1U')
('Viva Hate [2011 - Remaster] (2011 Remastered Version)', '6t4LdKTNWIOr2PZIB8tiZF')
('Katana', '56Xc3MPMf37InvPMBulRrU')
('E

In [34]:
top_genres = {}

def get_top_music(query_params={"limit": "50", "offset": "0", "time_range": "medium_term"}, 
                  type="artists", access_token=ACCESS_TOKEN):
    url = "https://api.spotify.com/v1/me/top/"
    if (type == "artists"):
        url += "artists"
    else:
        url += "tracks"
        
    headers = {'Authorization': "Bearer {}".format(ACCESS_TOKEN)}
    
    items = parsed['items']
    top_tracks_and_artists = requests.get(url=url, params={"limit": "50", "offset": "0", "time_range": "medium_term"}, headers={'Authorization': "Bearer {}".format(ACCESS_TOKEN)})
    parsed = json.loads(top_tracks_and_artists.text)
    items = parsed['items']

    if (type == "artists"):
        top_artists = []
        for i in range(len(items)):
            top_artists.append(["Rank: {}".format(str(i+1)), items[i]['name'], int(items[i]['followers']['total']), items[i]['images'][0]['url']])
        return top_artists
    else:
        top_tracks = []
        for i in range(len(items)):
            current_item = items[i]
            artists = current_item['artist']
            current_album = current_item['album']
            top_tracks.append(["Rank: {}".format(str(i+1)), current_item['name'], 
                               [artists[j]['name'] for j in range(len(artists))], 
                               current_album['name'], ])
for i in range(len(items)):
    top_artists.append


In [31]:
for item in top_artists:
    print(item)

['Rank: 1', 'Playboi Carti', 2572709, 'https://i.scdn.co/image/d3c4c2a06332c88e967387b38177d17d20e2331b']
['Rank: 2', 'Tory Lanez', 2422065, 'https://i.scdn.co/image/319b5b30148cb0614be70f115b272afc18ea69c7']
['Rank: 3', 'Khalid', 7773739, 'https://i.scdn.co/image/e52c824585bfeef4d674b509437120805300f557']
['Rank: 4', 'Travis Scott', 6219217, 'https://i.scdn.co/image/dc5eba5e032c2e5bc4d42c89d61ffee5875c981f']
['Rank: 5', 'Kid Cudi', 3177746, 'https://i.scdn.co/image/e69bd60976fd81d1368adc34423c691d86f92e0d']
['Rank: 6', 'Drake', 39481860, 'https://i.scdn.co/image/012ecd119617ac24ab56620ace4b81735b172686']
['Rank: 7', '6LACK', 1713840, 'https://i.scdn.co/image/b7b33aaeac1367b35d9c74a53fc9cfbd38f79fee']
['Rank: 8', 'NAV', 1501012, 'https://i.scdn.co/image/819c7e7c7126e367b35ff23e0653b1779069128f']
['Rank: 9', 'Chase Atlantic', 301103, 'https://i.scdn.co/image/8b9e716ede032187fb4a5b2b75d9c5cf2438c6e8']
['Rank: 10', 'The Weeknd', 16751255, 'https://i.scdn.co/image/a5fa0d35a5070867261d100b8

In [ ]:
"https://api.spotify.com/v1/me/top/artists"